In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_psremoting_stager.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','Channel']]

[(df['Channel'].isin(['Microsoft-Windows-PowerShell/Operational','Windows PowerShell']))
    & (df['EventID'].isin([400,4103]))
    & (df['Message'].str.contains('.*HostApplication.*wsmprovhost.*', regex=True))
]
.head()
)

,@timestamp,Hostname,Channel
678,2020-09-20T21:09:11.871Z,WORKSTATION6,Windows PowerShell


In [4]:
(
df[['@timestamp','Hostname','Application','SourceAddress','DestAddress','LayerName','LayerRTID']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5156)
    & (
        (df['DestPort'] == 5985)
        | (df['DestPort'] == 5986)
    )
    & (df['LayerRTID'] == 44)
]
.head()
)

,@timestamp,Hostname,Application,SourceAddress,DestAddress,LayerName,LayerRTID
478,2020-09-20T21:09:09.800Z,WORKSTATION6.theshire.local,System,172.18.39.5,172.18.39.6,%%14610,44.0
745,2020-09-20T21:09:11.902Z,WORKSTATION6.theshire.local,System,172.18.39.5,172.18.39.6,%%14610,44.0
856,2020-09-20T21:09:12.962Z,WORKSTATION6.theshire.local,System,172.18.39.5,172.18.39.6,%%14610,44.0


In [6]:
(
df[['@timestamp','Hostname','ParentProcessName','NewProcessName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4688)
    & (
        (df['ParentProcessName'].str.lower().str.endswith('wsmprovhost.exe', na=False))
        | (df['NewProcessName'].str.lower().str.endswith('wsmprovhost.exe', na=False))
    )
]
.head()
)

,@timestamp,Hostname,ParentProcessName,NewProcessName
535,2020-09-20T21:09:09.825Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wsmprovhost.exe
867,2020-09-20T21:09:12.968Z,WORKSTATION6.theshire.local,C:\Windows\System32\wsmprovhost.exe,C:\Windows\System32\WindowsPowerShell\v1.0\pow...


In [7]:
(
df[['@timestamp','Hostname','User','Initiated','Image','SourceIp','DestinationIp']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 3)
    & (
        (df['DestinationPort'] == 5985)
        | (df['DestinationPort'] == 5986)
    )
    & (~df['User'].isin(['NT AUTHORITY\\NETWORK SERVICE', 'NT AUTHORITY\\SYSTEM']))
]
)

,@timestamp,Hostname,User,Initiated,Image,SourceIp,DestinationIp
689,2020-09-20T21:09:11.877Z,WORKSTATION5.theshire.local,THESHIRE\pgustavo,true,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,172.18.39.5,172.18.39.6
1415,2020-09-20T21:09:13.263Z,WORKSTATION5.theshire.local,THESHIRE\pgustavo,true,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,172.18.39.5,172.18.39.6
1418,2020-09-20T21:09:13.264Z,WORKSTATION5.theshire.local,THESHIRE\pgustavo,true,C:\Windows\System32\WindowsPowerShell\v1.0\pow...,172.18.39.5,172.18.39.6
